# 01 - 環境驗證

驗證 Databricks 環境中 JAX + MuJoCo + MJX 是否正常運作。

**重要**：確保已通過 Cluster Library 安裝 `requirements.txt`

In [ ]:
# Cell 1: 環境變數設置
# ⚠️ 必須在 import mujoco 之前執行！
import os
os.environ["MUJOCO_GL"] = "disabled"  # Databricks 必須用 disabled，不是 osmesa
print(f"MUJOCO_GL = {os.environ.get('MUJOCO_GL')}")

In [ ]:
# Cell 2: JAX 驗證
import jax
import jaxlib

print(f"JAX version: {jax.__version__}")
print(f"jaxlib version: {jaxlib.__version__}")

# 檢查版本一致性
assert jax.__version__ == jaxlib.__version__, f"版本不一致！jax={jax.__version__}, jaxlib={jaxlib.__version__}"
print("✅ JAX/jaxlib 版本一致")

# 檢查 GPU 設備
devices = jax.devices()
print(f"\nDevices: {devices}")

if any('cuda' in str(d).lower() or 'gpu' in str(d).lower() for d in devices):
    print("✅ GPU 可用")
else:
    print("⚠️ 警告：未偵測到 GPU，將使用 CPU")

In [ ]:
# Cell 3: MuJoCo 驗證
import mujoco
print(f"MuJoCo version: {mujoco.__version__}")
print("✅ MuJoCo import 成功")

In [ ]:
# Cell 4: MJX 驗證
from mujoco import mjx
print("✅ MJX import 成功")

In [ ]:
# Cell 5: 簡單模型測試
xml_string = """
<mujoco model="test">
  <worldbody>
    <body name="ball" pos="0 0 1">
      <freejoint/>
      <geom type="sphere" size="0.1" mass="1"/>
    </body>
  </worldbody>
</mujoco>
"""

mj_model = mujoco.MjModel.from_xml_string(xml_string)
mj_data = mujoco.MjData(mj_model)

print(f"Bodies: {mj_model.nbody}")
print(f"Joints: {mj_model.njnt}")
print(f"qpos dim: {mj_model.nq}")
print("✅ MuJoCo 模型載入成功")

In [ ]:
# Cell 6: MJX GPU 編譯測試
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)

print(f"mjx_data.qpos shape: {mjx_data.qpos.shape}")
print(f"mjx_data.qpos dtype: {mjx_data.qpos.dtype}")
print("✅ MJX 編譯成功")

In [ ]:
# Cell 7: MJX step 測試
@jax.jit
def mjx_step(mjx_model, mjx_data):
    return mjx.step(mjx_model, mjx_data)

# 第一次 step（包含 JIT 編譯）
import time
t0 = time.time()
mjx_data_new = mjx_step(mjx_model, mjx_data)
mjx_data_new.qpos.block_until_ready()  # 等待 GPU 完成
t1 = time.time()
print(f"First step (with JIT): {(t1-t0)*1000:.2f} ms")

# 後續 step（純執行）
t0 = time.time()
for _ in range(100):
    mjx_data_new = mjx_step(mjx_model, mjx_data_new)
mjx_data_new.qpos.block_until_ready()
t1 = time.time()
print(f"100 steps: {(t1-t0)*1000:.2f} ms ({(t1-t0)*10:.4f} ms/step)")

print("\n✅ 環境驗證完成！")

## 驗證結果摘要

如果所有 cell 都顯示 ✅，環境設置正確。

### 故障排除

| 問題 | 解決方案 |
|------|----------|
| JAX/jaxlib 版本不一致 | 重新安裝 requirements.txt |
| 未偵測到 GPU | 確認 Cluster 有 GPU，檢查 `jax-cuda12-plugin` 是否安裝 |
| MuJoCo import 失敗 | 確認 `MUJOCO_GL=disabled` 在 import 前設置 |
| MJX import 失敗 | 確認 `mujoco-mjx` 已安裝 |